In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/guardian_articles.csv']

In [ ]:
# Load dataset
data_sample = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# Using only 10% of the data
data = data_sample.sample(frac=0.01, random_state=42)

# Drop rows with missing values
data.dropna(subset=['webTitle', 'bodyContent'], inplace=True)

In [ ]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for advanced text cleaning
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", " ", text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize text
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstruct the text from tokens
    text = ' '.join(tokens)

    return text

# Apply advanced text preprocessing
data['webTitle'] = data['webTitle'].apply(clean_text)
data['bodyContent'] = data['bodyContent'].apply(clean_text)

# Add <start> and <end> tokens to each headline
data['webTitle'] = data['webTitle'].apply(lambda x: '<start> ' + x + ' <end>')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
data['webTitle']

118646    <start> demand tory mp scrap covid rule could ...
124284    <start> really smart motorway would lower spee...
141061    <start> ‘profiting suffering’ ap cancel sale m...
80327     <start> school prison pipeline criminal justic...
96867     <start> naked coronavirus tale desperation mad...
                                ...                        
140283    <start> uk new zealand sign free trade deal <end>
110667    <start> crisis stage texas border city reel co...
1812      <start> apple iphone sale projected stagnant q...
91415      <start> woke gammon buzzword people coined <end>
31590     <start> arsène wenger say arsenal fa cup win a...
Name: webTitle, Length: 1486, dtype: object

In [ ]:
# Initialize tokenizer with a smaller vocabulary
max_vocab_size = 50  # Adjust as needed
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(data['webTitle'])
tokenizer.fit_on_texts(data['bodyContent'])

# Convert texts to sequences
sequences_body = tokenizer.texts_to_sequences(data['bodyContent'])
sequences_title = tokenizer.texts_to_sequences(data['webTitle'])

# Padding sequences
body_padded = pad_sequences(sequences_body, maxlen=300)  # Adjust maxlen as per your data
title_padded = pad_sequences(sequences_title, maxlen=30, padding='post')  # Adjust maxlen as per your data

# Preparing decoder input
title_padded_shifted = np.zeros_like(title_padded)
title_padded_shifted[:, 1:] = title_padded[:, :-1]

In [ ]:
# Splitting data into training and validation sets
X_train, X_val, title_train, title_val, decoder_input_train, decoder_input_val = train_test_split(
    body_padded, title_padded, title_padded_shifted, test_size=0.2, random_state=42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/guardian_articles.csv']

In [ ]:
# Load dataset
data_sample = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# Using only 10% of the data
data = data_sample.sample(frac=0.01, random_state=42)

# Drop rows with missing values
data.dropna(subset=['webTitle', 'bodyContent'], inplace=True)

In [ ]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for advanced text cleaning
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", " ", text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize text
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstruct the text from tokens
    text = ' '.join(tokens)

    return text

# Apply advanced text preprocessing
data['webTitle'] = data['webTitle'].apply(clean_text)
data['bodyContent'] = data['bodyContent'].apply(clean_text)

# Add <start> and <end> tokens to each headline
data['webTitle'] = data['webTitle'].apply(lambda x: '<start> ' + x + ' <end>')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
data['webTitle']

118646    <start> demand tory mp scrap covid rule could ...
124284    <start> really smart motorway would lower spee...
141061    <start> ‘profiting suffering’ ap cancel sale m...
80327     <start> school prison pipeline criminal justic...
96867     <start> naked coronavirus tale desperation mad...
                                ...                        
140283    <start> uk new zealand sign free trade deal <end>
110667    <start> crisis stage texas border city reel co...
1812      <start> apple iphone sale projected stagnant q...
91415      <start> woke gammon buzzword people coined <end>
31590     <start> arsène wenger say arsenal fa cup win a...
Name: webTitle, Length: 1486, dtype: object

In [ ]:
# Initialize tokenizer with a smaller vocabulary
max_vocab_size = 50  # Adjust as needed
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(data['webTitle'])
tokenizer.fit_on_texts(data['bodyContent'])

# Convert texts to sequences
sequences_body = tokenizer.texts_to_sequences(data['bodyContent'])
sequences_title = tokenizer.texts_to_sequences(data['webTitle'])

# Padding sequences
body_padded = pad_sequences(sequences_body, maxlen=300)  # Adjust maxlen as per your data
title_padded = pad_sequences(sequences_title, maxlen=30, padding='post')  # Adjust maxlen as per your data

# Preparing decoder input
title_padded_shifted = np.zeros_like(title_padded)
title_padded_shifted[:, 1:] = title_padded[:, :-1]

In [ ]:
# Splitting data into training and validation sets
X_train, X_val, title_train, title_val, decoder_input_train, decoder_input_val = train_test_split(
    body_padded, title_padded, title_padded_shifted, test_size=0.2, random_state=42)

In [ ]:
# Function to convert to sparse categorical
def to_sparse_categorical(y, num_classes):
    y_sparse = []
    for sequence in y:
        y_sparse_seq = np.zeros((len(sequence), num_classes))
        for i, idx in enumerate(sequence):
            if idx < num_classes:
                y_sparse_seq[i, idx] = 1
        y_sparse.append(y_sparse_seq)
    return np.array(y_sparse)

# Converting training and validation target data to sparse categorical
num_classes = max_vocab_size + 1  # +1 for the padding token
y_train_sparse = to_sparse_categorical(title_train, num_classes)
y_val_sparse = to_sparse_categorical(title_val, num_classes)

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Bidirectional

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=max_vocab_size + 1, output_dim=128)
encoder_lstm = Bidirectional(LSTM(128, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding(encoder_inputs))
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(input_dim=max_vocab_size + 1, output_dim=128)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_dense = Dense(max_vocab_size + 1, activation='softmax')  # Correct number of units

decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
model.summary()

In [ ]:
history = model.fit([X_train, decoder_input_train], y_train_sparse,
                    validation_data=([X_val, decoder_input_val], y_val_sparse),
                    epochs=5,  # Adjust as needed
                    batch_size=32)

In [ ]:
from keras.models import Sequential, load_model
import pickle
#saving model
model.save('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')
with open('/content/drive/My Drive/path/Models/seq2seq_headline_generator/history.p', 'wb') as f:
    pickle.dump(history.history, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
